<a href="https://colab.research.google.com/github/BetaUliansyah/Sandbox/blob/main/Monabsensi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Monabppk Wrapper

In [ ]:
#@Monabsensi by beta.uliansyah@pknstan.ac.id
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse 
import json
import re
from tabulate import tabulate

username = ''
password = ''


s = requests.Session()
r = s.get('https://monabppk.kemenkeu.go.id/')
bsoup = BeautifulSoup(r.text, 'html.parser')

if session.status_code==200:
    # mencari csrf token
    csrf_value = bsoup.find("input", {"name":"csrf_test_name"})['value']
    r = s.post('https://monabppk.kemenkeu.go.id/index.php/login', 
                           data={'csrf_test_name': csrf_value, 'inputEmail': username, 'inputPassword': password})
    #print(r.text)
    if r.status_code==200 and username in r.text:
        r = s.get('https://monabppk.kemenkeu.go.id/index.php/presensi')
        #print(r.text)
        datasoup = BeautifulSoup(r.text, 'html.parser')


        # menampilkan nama
        nama = datasoup.find("h3", {"class":"box-title pull-right"}).text
        print("Menampilkan data monabsensi atas nama " + nama)
    
        # menyiapkan tabel
        data = []
        table = datasoup.find('table', attrs={'class':'table table-bordered table-hover'})

        # membaca header table
        table_body = table.find('thead')
        rows = table_body.find_all('tr')
        for row in rows:
            cols = row.find_all('th')
            cols = [ele.text.strip() for ele in cols]
            data.append([ele for ele in cols if ele]) # Get rid of empty values

        # membaca table body
        table_body = table.find('tbody')
        rows = table_body.find_all('tr')
        for row in rows:
            cols = row.find_all('td')
            cols = [ele.text.strip() for ele in cols]
            data.append([ele for ele in cols if ele]) # Get rid of empty values
        print(tabulate(data))

# logout
r = s.get('https://monabppk.kemenkeu.go.id/index.php/login/logout')

# Credits
# https://stackoverflow.com/questions/23377533/python-beautifulsoup-parsing-table
# https://stackoverflow.com/questions/9535954/printing-lists-as-tabular-data

Menampilkan data monabsensi atas nama Beta Andri Anggiano Uliansyah - 197912062001121001
--  ------  ----------  ------------  -----  -------------  ------  ----------  -------------------  -----------  -----------
No  Hari    Tanggal     Jadwal Masuk  Masuk  Jadwal Pulang  Pulang  Cuti 1/2 H  Keterangan           Lama TL/PSW  Log Absensi
1   Rabu    1-04-2020   07:59         07:59  17:29          19:13   -           -                    -            ...
2   Kamis   2-04-2020   07:05         07:05  16:35          18:15   -           -                    -            ...
3   Jumat   3-04-2020   07:00         06:06  16:30          21:14   -           -                    -            ...
4   Sabtu   4-04-2020   00:00         00:00  00:00          00:00   -           Libur                -            ...
5   Minggu  5-04-2020   00:00         00:00  00:00          00:00   -           Libur                -            ...
6   Senin   6-04-2020   07:00         06:15  16:30          17:52   -

# Monabsensi Today

In [29]:
import requests
from bs4 import BeautifulSoup
import re
import datetime
import urllib3
import json

# Pushbullet
def pushbullet_message(title, body):
    msg = {"type": "note", "title": title, "body": body}
    TOKEN = 'o.689jKnqZumRxjvSUSFMGCHnrYV5xGKF0'
    resp = requests.post('https://api.pushbullet.com/v2/pushes', 
                         data=json.dumps(msg),
                         headers={'Authorization': 'Bearer ' + TOKEN,
                                  'Content-Type': 'application/json'})
    if resp.status_code != 200:
        raise Exception('Error',resp.status_code)
    else:
        print ('Message sent') 

session = None

def set_global_session():
    global session
    if not session:
        session = requests.Session()
        urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
        session.verify = False

def get_token():
    with session.get('https://monabppk.kemenkeu.go.id/') as response:
        if response.status_code==200 and len(response.text) != 0:
            bsoup = BeautifulSoup(response.text, 'html.parser')
            csrf_value = bsoup.find("input", {"name":"csrf_test_name"})['value']
            return csrf_value

def login(csrf_value, username, password):
    with session.post('https://monabppk.kemenkeu.go.id/index.php/login', 
                      data={'csrf_test_name': csrf_value, 'inputEmail': username, 'inputPassword': password}) as response:
        if response.status_code==200 and len(response.text) != 0:
            return True

def presensi():
    with session.get('https://monabppk.kemenkeu.go.id/index.php/mona') as response:
        if response.status_code==200 and len(response.text) != 0:
            bsoup = BeautifulSoup(response.text, 'html.parser')
            attend_links = bsoup.find_all('a', attrs = {'class' : 'small-box-footer'})
            for alink in attend_links:
                if "presensi" in alink['href']:
                    alink = alink['href']
                    #print(alink)
                    break
    if (alink):
        with session.get(alink) as response:
            if response.status_code==200 and len(response.text) != 0:
                return True
       

def get_data_presensi():
    with session.get('https://monabppk.kemenkeu.go.id/index.php/presensi') as response:
        if response.status_code==200 and len(response.text) != 0:
            bsoup = BeautifulSoup(response.text, 'html.parser')
            tgl_hari_ini = datetime.date.today().strftime("%-d-%m-%Y")
            hasil_presensi = bsoup.find('td', text = re.compile(tgl_hari_ini))
            hari = hasil_presensi.find_previous_sibling('td')
            jam_masuk = hasil_presensi.findNext('td').findNext('td')
            jam_pulang = jam_masuk.findNext('td').findNext('td')
            keterangan = jam_pulang.findNext('td').findNext('td')
            return [hari.text, tgl_hari_ini, jam_masuk.text, jam_pulang.text, keterangan.text]
        
def logout():
    session.get('https://monabppk.kemenkeu.go.id/index.php/login/logout')

#pushbullet_message("Cek Monabsensi", "Tgl "+tanggal+": Absen pagi jam "+jam_masuk.text+", absen pulang jam "+jam_pulang.text+". Keterangan: "+keterangan.text)

if __name__ == "__main__":
   set_global_session()
   token = get_token()
   if login(token, '', ''):
       #print(presensi())
       data = get_data_presensi()
   print(data)
   logout()

# Credits:
# https://stackoverflow.com/questions/9007653/how-to-find-tag-with-particular-text-with-beautiful-soup

['Selasa', '19-01-2021', '06:20', '00:00', 'PSW4']


In [18]:
response.status_code


NameError: ignored